In [1]:
cd /home/carlywolfbrandt/galvanize/assignments/csk_database

/home/carlywolfbrandt/galvanize/assignments/csk_database


In [2]:
from flatten_json import flatten_json

In [3]:
import pandas as pd
import numpy as np
import string
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns
import os
import requests
import json 
from tqdm import tqdm
from pandas.io.json import json_normalize

In [4]:
df = pd.read_csv('hydrocarbon_smiles_data.gz', compression='gzip', header=0, index_col = 0, sep=',', quotechar='"')

In [5]:
df.head()

,isosmiles,version_id,parent_id
1765,c1ccccc1,479848,479848
4483,Cc1ccccc1C,299961464,299961464
7089,CCc1ccccc1,475202,475202
8798,C1c2ccccc12,301251831,301251831
12266,CCc1ccccc1C,475997,475997


In [6]:
df.isosmiles.duplicated().sum()

0

In [7]:
df.count()

isosmiles     2135
version_id    2135
parent_id     2135
dtype: int64

In [8]:
pd.set_option('display.max_colwidth', -1)
df['image_url'] = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/' + df['isosmiles'] + '/PNG'
df.head()

,isosmiles,version_id,parent_id,image_url
1765,c1ccccc1,479848,479848,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/c1ccccc1/PNG
4483,Cc1ccccc1C,299961464,299961464,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/Cc1ccccc1C/PNG
7089,CCc1ccccc1,475202,475202,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/CCc1ccccc1/PNG
8798,C1c2ccccc12,301251831,301251831,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/C1c2ccccc12/PNG
12266,CCc1ccccc1C,475997,475997,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/CCc1ccccc1C/PNG


In [12]:
df.index = range(len(df.index))
directory='/home/carlywolfbrandt/galvanize/assignments/csk_database/images/'
for i in range(len(df)):
    r = requests.get(df['image_url'][i])
    with open(directory + str(df['version_id'][i])+".png", 'wb') as f:
                    f.write(r.content)

FileNotFoundError: [Errno 2] No such file or directory: '/home/carlywolfbrandt/galvanize/assignments/csk_database/images/479848.png'

In [9]:
pd.set_option('display.max_colwidth', -1)
df['json_url'] = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/' + df['isosmiles'] + '/JSON'
df.head()

,isosmiles,version_id,parent_id,image_url,json_url
1765,c1ccccc1,479848,479848,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/c1ccccc1/PNG,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/c1ccccc1/JSON
4483,Cc1ccccc1C,299961464,299961464,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/Cc1ccccc1C/PNG,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/Cc1ccccc1C/JSON
7089,CCc1ccccc1,475202,475202,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/CCc1ccccc1/PNG,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/CCc1ccccc1/JSON
8798,C1c2ccccc12,301251831,301251831,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/C1c2ccccc12/PNG,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/C1c2ccccc12/JSON
12266,CCc1ccccc1C,475997,475997,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/CCc1ccccc1C/PNG,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/CCc1ccccc1C/JSON


In [ ]:
df.index = range(len(df.index))
rw = 0
df_list = []
for name, row in df.iterrows():
    df_json = pd.DataFrame()
    r = requests.get(row['json_url'])
    if r.status_code != 200:
        df_list.append(row)
    else:
        try:
            flatten_data = flatten_json(r.json())
            df_json = pd.DataFrame.from_dict(flatten_data, orient='index')
            df_json =  pd.concat([df_json, row])
            df_list.append(df_json)
        except:
            df_list.append(row)
    rw += 1
    if rw%100 == 0:
        df_combined = pd.concat(df_list, axis=1).T
        df_combined.to_csv('combined_data.gz', compression='gzip')

/home/carlywolfbrandt/anaconda3/envs/dsi/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [17]:
df_combined.head()

,PC_Compounds_0_atoms_aid_0,PC_Compounds_0_atoms_aid_1,PC_Compounds_0_atoms_aid_10,PC_Compounds_0_atoms_aid_11,PC_Compounds_0_atoms_aid_12,PC_Compounds_0_atoms_aid_13,PC_Compounds_0_atoms_aid_14,PC_Compounds_0_atoms_aid_15,PC_Compounds_0_atoms_aid_16,PC_Compounds_0_atoms_aid_17,...,PC_Compounds_0_stereo_1_tetrahedral_bottom,PC_Compounds_0_stereo_1_tetrahedral_center,PC_Compounds_0_stereo_1_tetrahedral_parity,PC_Compounds_0_stereo_1_tetrahedral_top,PC_Compounds_0_stereo_1_tetrahedral_type,image_url,isosmiles,json_url,parent_id,version_id
361,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/Cc1ccccc1CC1CC=CC1/PNG,Cc1ccccc1CC1CC=CC1,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/Cc1ccccc1CC1CC=CC1/JSON,114420151,114420151
362,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/CC(=C)CC#Cc1ccccc1/PNG,CC(=C)CC#Cc1ccccc1,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/CC(=C)CC#Cc1ccccc1/JSON,114369395,114369395
0,1,2,11,12,13,14,15,16,17,18,...,NaN,NaN,NaN,NaN,NaN,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/Cc1ccc2CCC(=C)c2c1/PNG,Cc1ccc2CCC(=C)c2c1,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/Cc1ccc2CCC(=C)c2c1/JSON,107145097,107145097
364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/C#Cc1ccc(cc1)C1CC1/PNG,C#Cc1ccc(cc1)C1CC1,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/C#Cc1ccc(cc1)C1CC1/JSON,107065031,107065031
0,1,2,11,12,13,14,15,16,17,18,...,NaN,NaN,NaN,NaN,NaN,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/CCCCc1ccc2ccccc2c1/PNG,CCCCc1ccc2ccccc2c1,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/CCCCc1ccc2ccccc2c1/JSON,4311408,4311408


In [58]:
df_combined[[col for col in df_combined.columns if "sval" in col]+["json_url", "isosmiles"]].tail()

,json_url,isosmiles
2130,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/c12c3c4c5c1c1c6c7c2c2c8c3c3c9c4c4c%10c5c5c1c1c6c6c%11c%12c%13c%14c%15c%16c%17c%14c%14c%18c%13c%11c1c1c5c%10c5c(c%14c%10c%17c%11c%13c%16c%14c%16c%15c%12c%12c%16c(c2c7c6%12)c2c8c3c(c%13c%142)c2c9c4c5c%10c%112)c%181/JSON,c12c3c4c5c1c1c6c7c2c2c8c3c3c9c4c4c%10c5c5c1c1c6c6c%11c%12c%13c%14c%15c%16c%17c%14c%14c%18c%13c%11c1c1c5c%10c5c(c%14c%10c%17c%11c%13c%16c%14c%16c%15c%12c%12c%16c(c2c7c6%12)c2c8c3c(c%13c%142)c2c9c4c5c%10c%112)c%181
2131,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/[2H]C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])[2H]/JSON,[2H]C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])[2H]
2132,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/C1C2c3ccccc3C1C13c4c5c6c7c8c9c(c%10c%11c1c1c4c4c%12c5c5c6c6c8c8c%13c%14c%15c%16c%17c%18c%19c%20c%21c%22c%23c(c%14c%14c8c9c8c%10c9c%11c%10c1c(c%20c%10c%22c9c%23c%148)c4c%19c%12c%17c5c%16c6%13)C%151C4CC(c5ccccc45)C%18%211)C237/JSON,C1C2c3ccccc3C1C13c4c5c6c7c8c9c(c%10c%11c1c1c4c4c%12c5c5c6c6c8c8c%13c%14c%15c%16c%17c%18c%19c%20c%21c%22c%23c(c%14c%14c8c9c8c%10c9c%11c%10c1c(c%20c%10c%22c9c%23c%148)c4c%19c%12c%17c5c%16c6%13)C%151C4CC(c5ccccc45)C%18%211)C237
2133,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/C1C2c3ccccc3C1C13c4c5c6c7c8c9c(c%10c%11c1c1c%12c4c4c%13c5c5c7c7c%14c%15c%16c%17c%18c%19c%20c%21c%22c%23c%24c(c%25c%15c(c87)c7c9c%10c8c(c%24c9c%22c(c1c%11c89)c%21c%12c4c%20c%18c%13c5c%14%17)c%257)C%161C4CC(c5ccccc45)C%19%231)C236/JSON,C1C2c3ccccc3C1C13c4c5c6c7c8c9c(c%10c%11c1c1c%12c4c4c%13c5c5c7c7c%14c%15c%16c%17c%18c%19c%20c%21c%22c%23c%24c(c%25c%15c(c87)c7c9c%10c8c(c%24c9c%22c(c1c%11c89)c%21c%12c4c%20c%18c%13c5c%14%17)c%257)C%161C4CC(c5ccccc45)C%19%231)C236
2134,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/[2H]C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])[2H]/JSON,[2H]C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])C([2H])([2H])[2H]


In [42]:
df.iloc[361,:]

isosmiles     Cc1ccccc1CC1CC=CC1                                                               
version_id    114420151                                                                        
parent_id     114420151                                                                        
image_url     https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/Cc1ccccc1CC1CC=CC1/PNG 
json_url      https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/Cc1ccccc1CC1CC=CC1/JSON
Name: 361, dtype: object

In [23]:
df_combined.count()

PC_Compounds_0_atoms_aid_0             8 
PC_Compounds_0_atoms_aid_1             8 
PC_Compounds_0_atoms_aid_10            8 
PC_Compounds_0_atoms_aid_11            8 
PC_Compounds_0_atoms_aid_12            7 
PC_Compounds_0_atoms_aid_13            6 
PC_Compounds_0_atoms_aid_14            6 
PC_Compounds_0_atoms_aid_15            6 
PC_Compounds_0_atoms_aid_16            5 
PC_Compounds_0_atoms_aid_17            5 
PC_Compounds_0_atoms_aid_18            3 
PC_Compounds_0_atoms_aid_19            3 
PC_Compounds_0_atoms_aid_2             8 
PC_Compounds_0_atoms_aid_20            3 
PC_Compounds_0_atoms_aid_21            1 
PC_Compounds_0_atoms_aid_22            1 
PC_Compounds_0_atoms_aid_23            1 
PC_Compounds_0_atoms_aid_3             8 
PC_Compounds_0_atoms_aid_4             8 
PC_Compounds_0_atoms_aid_5             8 
PC_Compounds_0_atoms_aid_6             8 
PC_Compounds_0_atoms_aid_7             8 
PC_Compounds_0_atoms_aid_8             8 
PC_Compounds_0_atoms_aid_9        

In [22]:
df_combined[[col for col in df_combined.columns]+["json_url", "isosmiles"]].head()

,PC_Compounds_0_atoms_aid_0,PC_Compounds_0_atoms_aid_1,PC_Compounds_0_atoms_aid_10,PC_Compounds_0_atoms_aid_11,PC_Compounds_0_atoms_aid_12,PC_Compounds_0_atoms_aid_13,PC_Compounds_0_atoms_aid_14,PC_Compounds_0_atoms_aid_15,PC_Compounds_0_atoms_aid_16,PC_Compounds_0_atoms_aid_17,...,PC_Compounds_0_props_9_urn_source,PC_Compounds_0_props_9_urn_version,PC_Compounds_0_props_9_value_sval,image_url,isosmiles,json_url,parent_id,version_id,json_url,isosmiles
0,1,2,11,12,NaN,NaN,NaN,NaN,NaN,NaN,...,openeye.com,2.6.6,benzene,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/c1ccccc1/PNG,c1ccccc1,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/c1ccccc1/JSON,479848,479848,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/c1ccccc1/JSON,c1ccccc1
0,1,2,11,12,13,14,15,16,17,18,...,openeye.com,2.6.6,"1,2-xylene",https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/Cc1ccccc1C/PNG,Cc1ccccc1C,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/Cc1ccccc1C/JSON,299961464,299961464,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/Cc1ccccc1C/JSON,Cc1ccccc1C
0,1,2,11,12,13,14,15,16,17,18,...,openeye.com,2.6.6,ethylbenzene,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/CCc1ccccc1/PNG,CCc1ccccc1,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/CCc1ccccc1/JSON,475202,475202,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/CCc1ccccc1/JSON,CCc1ccccc1
0,1,2,11,12,13,NaN,NaN,NaN,NaN,NaN,...,openeye.com,2.6.6,"bicyclo[4.1.0]hepta-1,3,5-triene",https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/C1c2ccccc12/PNG,C1c2ccccc12,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/C1c2ccccc12/JSON,301251831,301251831,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/C1c2ccccc12/JSON,C1c2ccccc12
0,1,2,11,12,13,14,15,16,17,18,...,openeye.com,2.6.6,1-ethyl-2-methylbenzene,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/CCc1ccccc1C/PNG,CCc1ccccc1C,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/CCc1ccccc1C/JSON,475997,475997,https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/CCc1ccccc1C/JSON,CCc1ccccc1C


In [57]:
df.json_url.head()

0    https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/c1ccccc1/JSON   
1    https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/Cc1ccccc1C/JSON 
2    https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/CCc1ccccc1/JSON 
3    https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/C1c2ccccc12/JSON
4    https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/CCc1ccccc1C/JSON
Name: json_url, dtype: object

In [47]:
df_combined.set_index('PC_Compounds_0_props_9_value_sval')

,PC_Compounds_0_atoms_aid_0,PC_Compounds_0_atoms_aid_1,PC_Compounds_0_atoms_aid_10,PC_Compounds_0_atoms_aid_11,PC_Compounds_0_atoms_aid_12,PC_Compounds_0_atoms_aid_13,PC_Compounds_0_atoms_aid_14,PC_Compounds_0_atoms_aid_15,PC_Compounds_0_atoms_aid_16,PC_Compounds_0_atoms_aid_17,...,PC_Compounds_0_props_8_urn_source,PC_Compounds_0_props_8_urn_version,PC_Compounds_0_props_8_value_sval,PC_Compounds_0_props_9_urn_datatype,PC_Compounds_0_props_9_urn_label,PC_Compounds_0_props_9_urn_name,PC_Compounds_0_props_9_urn_release,PC_Compounds_0_props_9_urn_software,PC_Compounds_0_props_9_urn_source,PC_Compounds_0_props_9_urn_version
PC_Compounds_0_props_9_value_sval,,,,,,,,,,,,,,,,,,,,,
benzene,1,2,11,12,NaN,NaN,NaN,NaN,NaN,NaN,...,openeye.com,2.6.6,benzene,1,IUPAC Name,Preferred,2019.04.19,LexiChem,openeye.com,2.6.6
"1,2-xylene",1,2,11,12,13,14,15,16,17,18,...,openeye.com,2.6.6,"1,2-xylene",1,IUPAC Name,Preferred,2019.04.19,LexiChem,openeye.com,2.6.6
ethylbenzene,1,2,11,12,13,14,15,16,17,18,...,openeye.com,2.6.6,ethylbenzene,1,IUPAC Name,Preferred,2019.04.19,LexiChem,openeye.com,2.6.6
"bicyclo[4.1.0]hepta-1,3,5-triene",1,2,11,12,13,NaN,NaN,NaN,NaN,NaN,...,openeye.com,2.6.6,"bicyclo[4.1.0]hepta-1,3,5-triene",1,IUPAC Name,Preferred,2019.04.19,LexiChem,openeye.com,2.6.6
1-ethyl-2-methylbenzene,1,2,11,12,13,14,15,16,17,18,...,openeye.com,2.6.6,1-ethyl-2-methylbenzene,1,IUPAC Name,Preferred,2019.04.19,LexiChem,openeye.com,2.6.6
propylbenzene,1,2,11,12,13,14,15,16,17,18,...,openeye.com,2.6.6,propylbenzene,1,IUPAC Name,Preferred,2019.04.19,LexiChem,openeye.com,2.6.6
styrene,1,2,11,12,13,14,15,16,NaN,NaN,...,openeye.com,2.6.6,styrene,1,IUPAC Name,Preferred,2019.04.19,LexiChem,openeye.com,2.6.6
styrene,1,2,11,12,13,14,15,16,NaN,NaN,...,openeye.com,2.6.6,styrene,1,IUPAC Name,Preferred,2019.04.19,LexiChem,openeye.com,2.6.6
styrene,1,2,11,12,13,14,15,16,NaN,NaN,...,openeye.com,2.6.6,styrene,1,IUPAC Name,Preferred,2019.04.19,LexiChem,openeye.com,2.6.6


In [72]:
df_combined = df_combined.rename({"PC_Compounds_0_props_10_value_sval":"systematic_name"}, axis=1)

In [73]:
df_combined["systematic_name"]

0    benzene                         
0    1,2-dimethylbenzene             
0    ethylbenzene                    
0    bicyclo[4.1.0]hepta-1,3,5-triene
0    1-ethyl-2-methyl-benzene        
0    propylbenzene                   
0    ethenylbenzene                  
7    NaN                             
0    ethenylbenzene                  
8    NaN                             
0    ethenylbenzene                  
0    1-methyl-2-propyl-benzene       
Name: systematic_name, dtype: object